# The Bernstein Vazirani problem 

In [ ]:
import matplotlib.pyplot as plt 
from equal1.benchmarking.algorithms import deutschjozsa 
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
hardware = {'hpc-conservative':"Current Device extrapolated", 'hpc-gamma1': "32 Qubit QPU", 'hpc-gamma2': "167 Qubit QPU"}

In [ ]:
all_experiments = [deutschjozsa.BernsteinVazirani(n_qubits=n_q, device_name=hw, number_of_random_strings_to_test = min(2**n_q, 10), rng=7331) for n_q in range(3, 21, 2) for hw in hardware.keys()]
results = []
for individual in all_experiments:
    individual.run(simulate=True,  runtime_options={"backend":"TensorNet_MPS", "simulation_platform":"GPU"})
    individual.collect_results()
    anal = individual.analyse_results()
    # print(anal)
    for state, value in anal.items():
        row = {"n_qubits": individual.n_qubits, "device": individual.device_name, "state": int(state,2), "probability": value}
        results.append(row)
    print(f"Running for {individual.n_qubits} qubits {individual.device_name}, results: {anal}")



In [ ]:
df = pd.DataFrame.from_dict(results)
df.to_csv("bernstein_vazirani.csv")

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))
df["label"] = df["device"].map(hardware)
sns.lineplot(data=df, x="n_qubits", y="probability", hue="label", ax=ax)
ax.set_ylim(0, 1)
ax.grid(True)
ax.set_ylabel("Probability of measuring the hidden string")
ax.set_xlabel("Number of Qubits")
ax.legend(title="Simulated Devices")
ax.set_title("Bernstein-Vazirani Problem Performance")
